<a href="https://colab.research.google.com/github/alinaalborova/russian_idioms_processing/blob/main/MICE_Token_Level_ruBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Idiom Type and Token Classification

Based on [MICE: Mining Idioms with Contextual Embeddings](https://arxiv.org/pdf/2008.05759.pdf) by  Škvorc et al.


## Libraries

In [ ]:
!pip install transformers
!pip install tensor2tensor

In [ ]:
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Activation, Concatenate, Masking, GRU
from tensorflow.keras import Sequential
from tensorflow.keras import Input, Model
from tensorflow.keras.utils import to_categorical
from ast import literal_eval
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
#from keras.utils import plot_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import tensorflow as tf
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm_notebook

## Dataset

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset_vnc_dir = '/content/drive/MyDrive/ВКР/Sense Disambiguation Corpus/token_level_vnc.csv'
data_vnc = pd.read_csv(dataset_vnc_dir )
data_vnc.drop(data_vnc.iloc[:, :1], axis=1, inplace=True)
data_vnc.head(2)

,Idiom Normal,Idiom Inflected,Token,Label,Context
0,взваливать на плечи,взваливают на плечи,бесчеловечный,2,"Там несвобода, и тут несвобода, там сажают и ..."
1,выступить на сцену,выступить на сцену,выступить,0,"Monsieur Орлов, наш учитель пения, говорил мн..."


In [ ]:
dataset_anc_dir = '/content/drive/MyDrive/ВКР/Sense Disambiguation Corpus/token_level_anc.csv'
data_anc = pd.read_csv(dataset_anc_dir )
data_anc.drop(data_anc.iloc[:, :1], axis=1, inplace=True)
data_anc.head(2)

,Idiom Normal,Idiom Inflected,Token,Label,Context
0,старый воробей,старые воробьи,старые,1,"― Ведь мы с вами, Марья Алексевна, старые вор..."
1,заблудшая овца,заблудшим овцам,заблудшим,1,"― Хирлемс не должен быть слишком строг, если..."


## Define Classifier

In [ ]:
MAX_CONTEXT_LEN = 149
MAX_TOKEN_LEN = 13
VECTOR_DIM = 768
NUM_CLASSES = 3

def build_model():
    input_token = Input(shape=(MAX_TOKEN_LEN, VECTOR_DIM), name='input1')
    input_context = Input(shape=(MAX_CONTEXT_LEN,VECTOR_DIM), name='input2')

    #token
    forward_layer = GRU(10, return_sequences=False, dropout=0.5)
    backward_layer = GRU(10, return_sequences=False, dropout=0.5,
                    go_backwards=True)
    bidirectional1 = Bidirectional(forward_layer, backward_layer=backward_layer,
                      input_shape=(MAX_TOKEN_LEN,VECTOR_DIM))(input_token)

    #context
    bidirectional2 = Bidirectional(forward_layer, backward_layer=backward_layer,
                      input_shape=(MAX_CONTEXT_LEN,VECTOR_DIM))(input_context)

    concat = Concatenate(axis=1)([bidirectional1, bidirectional2])

    dense = Dense(NUM_CLASSES)(concat)
    softmax = Activation('softmax', name='output')(dense)

    model = Model(inputs=[input_token, input_context], outputs=softmax)
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    #plot_model(model, to_file='multiple_inputs.png')
    return model
model_all = build_model()

model_all.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input1 (InputLayer)             [(None, 13, 768)]    0                                            
__________________________________________________________________________________________________
input2 (InputLayer)             [(None, 149, 768)]   0                                            
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 20)           46800       input1[0][0]                     
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 20)           46800       input2[0][0]                     
______________________________________________________________________________________________

## Embed

### Load BERT

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel, TFAutoModel
  
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

BERT_model = AutoModel.from_pretrained('DeepPavlov/rubert-base-cased')

In [ ]:
def get_bert_embeddings_single_sentence(sentence):
  inputs = tokenizer(sentence, return_tensors="pt")
  outputs=BERT_model(**inputs)
  return outputs.last_hidden_state
  
def get_bert_embeddings(sentences, padded_size=150, vec_size=768):
  embeddings_all = []
  #for sentence in tqdm_notebook(sentences):
  for sentence in tqdm_notebook(sentences):
    lhs = get_bert_embeddings_single_sentence(sentence)
    vecs = lhs[0]
    sentence_padded = np.empty(shape=(padded_size, ), dtype=object)
    for id, token in enumerate(sentence_padded):
      try:
        sentence_padded[id] = vecs[id].detach().numpy()
      except:
        sentence_padded[id] = np.zeros(vec_size)
    embeddings_all.append(sentence_padded)
  return embeddings_all

## VNC

In [ ]:
tokens_vnc = get_bert_embeddings(data_vnc['Token'][:2000], padded_size=15)

In [ ]:
tokens_vnc2 = get_bert_embeddings(data_vnc['Token'][2000:], padded_size=15)

In [ ]:
embedded_tokens_vnc = tokens_vnc + tokens_vnc2

In [ ]:
contexts_vnc1 = get_bert_embeddings(data_vnc['Context'][:2000])

In [ ]:
contexts_vnc2 = get_bert_embeddings(data_vnc['Context'][2000:])

In [ ]:
embedded_contexts_vnc = contexts_vnc1 + contexts_vnc2

In [ ]:
labels_vnc = to_categorical(data_vnc.Label)
X_train_tokens_vnc, X_test_tokens_vnc = train_test_split(embedded_tokens_vnc, test_size=0.3, random_state=34)
X_train_contexts_vnc, X_test_contexts_vnc = train_test_split(embedded_contexts_vnc, test_size=0.3, random_state=34)
Y_train_vnc, Y_test_vnc = train_test_split(labels_vnc, test_size=0.3, random_state=34)

### Classifier

#### Define

In [ ]:
MAX_CONTEXT_LEN = 150
MAX_TOKEN_LEN = 15
VECTOR_DIM = 768
NUM_CLASSES = 3

def build_model():
    input_token = Input(shape=(MAX_TOKEN_LEN, VECTOR_DIM), name='input1')
    input_context = Input(shape=(MAX_CONTEXT_LEN,VECTOR_DIM), name='input2')

    #token
    forward_layer = GRU(10, return_sequences=False, dropout=0.5)
    backward_layer = GRU(10, return_sequences=False, dropout=0.5,
                    go_backwards=True)
    bidirectional1 = Bidirectional(forward_layer, backward_layer=backward_layer,
                      input_shape=(MAX_TOKEN_LEN,VECTOR_DIM))(input_token)

    #context
    bidirectional2 = Bidirectional(forward_layer, backward_layer=backward_layer,
                      input_shape=(MAX_CONTEXT_LEN,VECTOR_DIM))(input_context)

    concat = Concatenate(axis=1)([bidirectional1, bidirectional2])

    dense = Dense(NUM_CLASSES)(concat)
    softmax = Activation('softmax', name='output')(dense)

    model = Model(inputs=[input_token, input_context], outputs=softmax)
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    #plot_model(model, to_file='multiple_inputs.png')
    return model

#### Train & Test

In [ ]:
model_vnc = build_model()

In [ ]:
tokens_tensor_train_vnc = tf.convert_to_tensor(X_train_tokens_vnc)
contexts_tensor_train_vnc = tf.convert_to_tensor(X_train_contexts_vnc)
tokens_tensor_test_vnc = tf.convert_to_tensor(X_test_tokens_vnc)
contexts_tensor_test_vnc = tf.convert_to_tensor(X_test_contexts_vnc)

In [ ]:
model_vnc.fit({'input1': tokens_tensor_train_vnc, 'input2': contexts_tensor_train_vnc}, 
               {'output': Y_train_vnc}, batch_size=8, epochs=10)

Epoch 1/10
337/337 [==============================] - 45s 115ms/step - loss: 0.5359 - accuracy: 0.5776
Epoch 2/10
337/337 [==============================] - 39s 116ms/step - loss: 0.3741 - accuracy: 0.7585
Epoch 3/10
337/337 [==============================] - 39s 115ms/step - loss: 0.3043 - accuracy: 0.8117
Epoch 4/10
337/337 [==============================] - 38s 113ms/step - loss: 0.2740 - accuracy: 0.8306
Epoch 5/10
337/337 [==============================] - 38s 114ms/step - loss: 0.2519 - accuracy: 0.8507
Epoch 6/10
337/337 [==============================] - 39s 115ms/step - loss: 0.2294 - accuracy: 0.8692
Epoch 7/10
337/337 [==============================] - 38s 114ms/step - loss: 0.2188 - accuracy: 0.8711
Epoch 8/10
337/337 [==============================] - 38s 114ms/step - loss: 0.2048 - accuracy: 0.8830
Epoch 9/10
337/337 [==============================] - 39s 115ms/step - loss: 0.2016 - accuracy: 0.8826
Epoch 10/10
337/337 [==============================] - 39s 115ms/step - l

In [ ]:
model_vnc.evaluate({'input1': tokens_tensor_test_vnc, 'input2': contexts_tensor_test_vnc}, 
               {'output': Y_test_vnc})

37/37 [==============================] - 3s 53ms/step - loss: 0.2163 - accuracy: 0.8804


[0.21632592380046844, 0.8804159164428711]

In [ ]:
preds_vnc = model_vnc.predict({'input1': tokens_tensor_test_vnc, 'input2': contexts_tensor_test_vnc})
f1_score(np.argmax(preds_vnc, axis=1), np.argmax(Y_test_vnc, axis=1), average='micro')

0.8804159445407279

In [ ]:
f1_score(np.argmax(preds_vnc, axis=1), np.argmax(Y_test_vnc, axis=1), average='macro')

0.8742530924506257

In [ ]:
f1_score(np.argmax(preds_vnc, axis=1), np.argmax(Y_test_vnc, axis=1), average='weighted')

0.878672066300463

## ANC

In [ ]:
data_anc.shape

(2540, 5)

In [ ]:
tokens_anc = get_bert_embeddings(data_anc['Token'][:1000], padded_size=15)
tokens_anc2 = get_bert_embeddings(data_anc['Token'][1000:], padded_size=15)

In [ ]:
embedded_tokens_anc = tokens_anc + tokens_anc2

In [ ]:
del tokens_anc
del tokens_anc2

In [ ]:
contexts_anc1 = get_bert_embeddings(data_anc['Context'][:1000])
contexts_anc2 = get_bert_embeddings(data_anc['Context'][1000:])

In [ ]:
embedded_contexts_anc = contexts_anc1 + contexts_anc2

In [ ]:
del contexts_anc1
del contexts_anc2

In [ ]:
labels_anc = to_categorical(data_anc.Label)
X_train_tokens_anc, X_test_tokens_anc = train_test_split(embedded_tokens_anc, test_size=0.3, random_state=34)
X_train_contexts_anc, X_test_contexts_anc = train_test_split(embedded_contexts_anc, test_size=0.3, random_state=34)
Y_train_anc, Y_test_anc = train_test_split(labels_anc, test_size=0.3, random_state=34)

### Classifier

#### Train & Test

In [ ]:
model_anc = build_model()

In [ ]:
tokens_tensor_train_anc = tf.convert_to_tensor(X_train_tokens_anc)
contexts_tensor_train_anc = tf.convert_to_tensor(X_train_contexts_anc)
tokens_tensor_test_anc = tf.convert_to_tensor(X_test_tokens_anc)
contexts_tensor_test_anc = tf.convert_to_tensor(X_test_contexts_anc)

In [ ]:
model_anc.fit({'input1': tokens_tensor_train_anc, 'input2': contexts_tensor_train_anc}, 
               {'output': Y_train_anc}, batch_size=8, epochs=10)

Epoch 1/10
223/223 [==============================] - 31s 112ms/step - loss: 0.5567 - accuracy: 0.5512
Epoch 2/10
223/223 [==============================] - 25s 111ms/step - loss: 0.4131 - accuracy: 0.7165
Epoch 3/10
223/223 [==============================] - 25s 112ms/step - loss: 0.3405 - accuracy: 0.7756
Epoch 4/10
223/223 [==============================] - 25s 111ms/step - loss: 0.3065 - accuracy: 0.8054
Epoch 5/10
223/223 [==============================] - 24s 109ms/step - loss: 0.2823 - accuracy: 0.8223
Epoch 6/10
223/223 [==============================] - 24s 108ms/step - loss: 0.2697 - accuracy: 0.8307
Epoch 7/10
223/223 [==============================] - 25s 112ms/step - loss: 0.2401 - accuracy: 0.8543
Epoch 8/10
223/223 [==============================] - 24s 110ms/step - loss: 0.2306 - accuracy: 0.8600
Epoch 9/10
223/223 [==============================] - 25s 110ms/step - loss: 0.2095 - accuracy: 0.8763
Epoch 10/10
223/223 [==============================] - 24s 109ms/step - l

In [ ]:
model_anc.evaluate({'input1': tokens_tensor_test_anc, 'input2': contexts_tensor_test_anc}, 
               {'output': Y_test_anc})

24/24 [==============================] - 3s 50ms/step - loss: 0.2543 - accuracy: 0.8465


[0.2543363571166992, 0.8464567065238953]

In [ ]:
preds_anc = model_anc.predict({'input1': tokens_tensor_test_anc, 'input2': contexts_tensor_test_anc})
f1_score(np.argmax(preds_anc, axis=1), np.argmax(Y_test_anc, axis=1), average='micro')

0.846456692913386

In [ ]:
f1_score(np.argmax(preds_anc, axis=1), np.argmax(Y_test_anc, axis=1), average='macro')

0.8282772401349909

In [ ]:
f1_score(np.argmax(preds_anc, axis=1), np.argmax(Y_test_anc, axis=1), average='weighted')

0.8440899316515293

### Smaller Dataset

#### 70%

In [ ]:
model_anc_70 = build_model()

In [ ]:
len(X_train_tokens_anc) * 0.7

1244.6

In [ ]:
len(X_test_tokens_anc) * 0.7

533.4

In [ ]:
tokens_tensor_train_anc_70 = tf.convert_to_tensor(X_train_tokens_anc[:1245])
tokens_tensor_test_anc_70 = tf.convert_to_tensor(X_test_tokens_anc[:533])
contexts_tensor_train_anc_70 = tf.convert_to_tensor(X_train_contexts_anc[:1245])
contexts_tensor_test_anc_70 = tf.convert_to_tensor(X_test_contexts_anc[:533])

In [ ]:
model_anc_70.fit({'input1': tokens_tensor_train_anc_70, 'input2': contexts_tensor_train_anc_70}, 
               {'output': Y_train_anc[:1245]}, batch_size=8, epochs=10)

Epoch 1/10
156/156 [==============================] - 24s 111ms/step - loss: 0.5827 - accuracy: 0.5221
Epoch 2/10
156/156 [==============================] - 17s 110ms/step - loss: 0.4626 - accuracy: 0.6635
Epoch 3/10
156/156 [==============================] - 17s 108ms/step - loss: 0.3717 - accuracy: 0.7494
Epoch 4/10
156/156 [==============================] - 17s 110ms/step - loss: 0.3369 - accuracy: 0.7807
Epoch 5/10
156/156 [==============================] - 17s 109ms/step - loss: 0.2963 - accuracy: 0.8072
Epoch 6/10
156/156 [==============================] - 17s 110ms/step - loss: 0.2798 - accuracy: 0.8313
Epoch 7/10
156/156 [==============================] - 17s 111ms/step - loss: 0.2589 - accuracy: 0.8514
Epoch 8/10
156/156 [==============================] - 17s 112ms/step - loss: 0.2484 - accuracy: 0.8450
Epoch 9/10
156/156 [==============================] - 17s 109ms/step - loss: 0.2281 - accuracy: 0.8554
Epoch 10/10
156/156 [==============================] - 17s 109ms/step - l

In [ ]:
model_anc_70.evaluate({'input1': tokens_tensor_test_anc_70, 'input2': contexts_tensor_test_anc_70}, 
               {'output': Y_test_anc[:533]})

17/17 [==============================] - 2s 48ms/step - loss: 0.3474 - accuracy: 0.7692


[0.3474300503730774, 0.7692307829856873]

In [ ]:
model_anc_70.evaluate({'input1': tokens_tensor_test_anc, 'input2': contexts_tensor_test_anc}, 
               {'output': Y_test_anc})

24/24 [==============================] - 1s 48ms/step - loss: 0.3370 - accuracy: 0.7743


[0.33695778250694275, 0.7742782235145569]

In [ ]:
preds_anc = model_anc_70.predict({'input1': tokens_tensor_test_anc_70, 'input2': contexts_tensor_test_anc_70})
f1_score(np.argmax(preds_anc, axis=1), np.argmax(Y_test_anc[:533], axis=1), average='micro')

0.7692307692307693

In [ ]:
f1_score(np.argmax(preds_anc, axis=1), np.argmax(Y_test_anc[:533], axis=1), average='macro')

0.7551143207017397

In [ ]:
f1_score(np.argmax(preds_anc, axis=1), np.argmax(Y_test_anc[:533], axis=1), average='weighted')

0.7608885368190298

In [ ]:
preds_anc = model_anc_70.predict({'input1': tokens_tensor_test_anc, 'input2': contexts_tensor_test_anc})
print(f1_score(np.argmax(preds_anc, axis=1), np.argmax(Y_test_anc, axis=1), average='micro'))
print(f1_score(np.argmax(preds_anc, axis=1), np.argmax(Y_test_anc, axis=1), average='macro'))
print(f1_score(np.argmax(preds_anc, axis=1), np.argmax(Y_test_anc, axis=1), average='weighted'))

0.7742782152230971
0.7575736606935134
0.7657849693883884


#### 50% 

In [ ]:
model_anc_50 = build_model()

In [ ]:
len(X_train_tokens_anc) * 0.5

889.0

In [ ]:
len(X_test_tokens_anc) * 0.5

381.0

In [ ]:
tokens_tensor_train_anc_50 = tf.convert_to_tensor(X_train_tokens_anc[:889])
tokens_tensor_test_anc_50 = tf.convert_to_tensor(X_test_tokens_anc[:381])
contexts_tensor_train_anc_50 = tf.convert_to_tensor(X_train_contexts_anc[:889])
contexts_tensor_test_anc_50 = tf.convert_to_tensor(X_test_contexts_anc[:381])

In [ ]:
model_anc_50.fit({'input1': tokens_tensor_train_anc_50, 'input2': contexts_tensor_train_anc_50}, 
               {'output': Y_train_anc[:889]}, batch_size=8, epochs=10)

Epoch 1/10
112/112 [==============================] - 18s 110ms/step - loss: 0.5843 - accuracy: 0.5231
Epoch 2/10
112/112 [==============================] - 12s 111ms/step - loss: 0.4896 - accuracy: 0.6457
Epoch 3/10
112/112 [==============================] - 12s 109ms/step - loss: 0.4011 - accuracy: 0.7379
Epoch 4/10
112/112 [==============================] - 12s 111ms/step - loss: 0.3412 - accuracy: 0.7919
Epoch 5/10
112/112 [==============================] - 12s 109ms/step - loss: 0.3116 - accuracy: 0.8200
Epoch 6/10
112/112 [==============================] - 12s 110ms/step - loss: 0.2824 - accuracy: 0.8200
Epoch 7/10
112/112 [==============================] - 12s 110ms/step - loss: 0.2620 - accuracy: 0.8414
Epoch 8/10
112/112 [==============================] - 13s 112ms/step - loss: 0.2288 - accuracy: 0.8763
Epoch 9/10
112/112 [==============================] - 12s 109ms/step - loss: 0.2184 - accuracy: 0.8864
Epoch 10/10
112/112 [==============================] - 12s 108ms/step - l

In [ ]:
model_anc_50.evaluate({'input1': tokens_tensor_test_anc_50, 'input2': contexts_tensor_test_anc_50}, 
               {'output': Y_test_anc[:381]})

12/12 [==============================] - 2s 53ms/step - loss: 0.3153 - accuracy: 0.7953


[0.3153291344642639, 0.7952755689620972]

In [ ]:
model_anc_50.evaluate({'input1': tokens_tensor_test_anc, 'input2': contexts_tensor_test_anc}, 
               {'output': Y_test_anc})

24/24 [==============================] - 1s 50ms/step - loss: 0.3237 - accuracy: 0.7861


[0.3237176835536957, 0.7860892415046692]

In [ ]:
preds_anc = model_anc_50.predict({'input1': tokens_tensor_test_anc_50, 'input2': contexts_tensor_test_anc_50})
f1_score(np.argmax(preds_anc, axis=1), np.argmax(Y_test_anc[:381], axis=1), average='micro')

0.7952755905511811

In [ ]:
f1_score(np.argmax(preds_anc, axis=1), np.argmax(Y_test_anc[:381], axis=1), average='macro')

0.7598666577325114

In [ ]:
f1_score(np.argmax(preds_anc, axis=1), np.argmax(Y_test_anc[:381], axis=1), average='weighted')

0.7974725046095319

In [ ]:
preds_anc = model_anc_50.predict({'input1': tokens_tensor_test_anc, 'input2': contexts_tensor_test_anc})
print(f1_score(np.argmax(preds_anc, axis=1), np.argmax(Y_test_anc, axis=1), average='micro'))
print(f1_score(np.argmax(preds_anc, axis=1), np.argmax(Y_test_anc, axis=1), average='macro'))
print(f1_score(np.argmax(preds_anc, axis=1), np.argmax(Y_test_anc, axis=1), average='weighted'))

0.7860892388451444
0.753947509573183
0.7885989264285002


#### 30%

In [ ]:
model_anc_30 = build_model()

In [ ]:
len(X_train_tokens_anc) * 0.3

533.4

In [ ]:
len(X_test_tokens_anc) * 0.3

228.6

In [ ]:
tokens_tensor_train_anc_30 = tf.convert_to_tensor(X_train_tokens_anc[:533])
tokens_tensor_test_anc_30 = tf.convert_to_tensor(X_test_tokens_anc[:228])
contexts_tensor_train_anc_30 = tf.convert_to_tensor(X_train_contexts_anc[:533])
contexts_tensor_test_anc_30 = tf.convert_to_tensor(X_test_contexts_anc[:228])

In [ ]:
model_anc_30.fit({'input1': tokens_tensor_train_anc_30, 'input2': contexts_tensor_train_anc_30}, 
               {'output': Y_train_anc[:533]}, batch_size=8, epochs=10)

Epoch 1/10
67/67 [==============================] - 14s 111ms/step - loss: 0.5990 - accuracy: 0.5216
Epoch 2/10
67/67 [==============================] - 7s 109ms/step - loss: 0.5495 - accuracy: 0.5591
Epoch 3/10
67/67 [==============================] - 7s 109ms/step - loss: 0.4917 - accuracy: 0.6398
Epoch 4/10
67/67 [==============================] - 7s 109ms/step - loss: 0.4201 - accuracy: 0.7186
Epoch 5/10
67/67 [==============================] - 7s 108ms/step - loss: 0.3818 - accuracy: 0.7505
Epoch 6/10
67/67 [==============================] - 7s 106ms/step - loss: 0.3440 - accuracy: 0.7974
Epoch 7/10
67/67 [==============================] - 7s 107ms/step - loss: 0.3152 - accuracy: 0.8011
Epoch 8/10
67/67 [==============================] - 7s 108ms/step - loss: 0.2955 - accuracy: 0.8311
Epoch 9/10
67/67 [==============================] - 7s 110ms/step - loss: 0.2821 - accuracy: 0.8405
Epoch 10/10
67/67 [==============================] - 7s 109ms/step - loss: 0.2566 - accuracy: 0.878

In [ ]:
model_anc_30.evaluate({'input1': tokens_tensor_test_anc_30, 'input2': contexts_tensor_test_anc_30}, 
               {'output': Y_test_anc[:228]})

8/8 [==============================] - 2s 43ms/step - loss: 0.3739 - accuracy: 0.7149


[0.37392914295196533, 0.7149122953414917]

In [ ]:
model_anc_30.evaluate({'input1': tokens_tensor_test_anc, 'input2': contexts_tensor_test_anc}, 
               {'output': Y_test_anc})

24/24 [==============================] - 1s 50ms/step - loss: 0.3808 - accuracy: 0.7231


[0.3807950019836426, 0.7230970859527588]

In [ ]:
preds_anc = model_anc_30.predict({'input1': tokens_tensor_test_anc_30, 'input2': contexts_tensor_test_anc_30})
f1_score(np.argmax(preds_anc, axis=1), np.argmax(Y_test_anc[:228], axis=1), average='micro')

0.7149122807017545

In [ ]:
f1_score(np.argmax(preds_anc, axis=1), np.argmax(Y_test_anc[:228], axis=1), average='macro')

0.6542285484916788

In [ ]:
f1_score(np.argmax(preds_anc, axis=1), np.argmax(Y_test_anc[:228], axis=1), average='weighted')

0.7115881047138798

In [ ]:
preds_anc = model_anc_30.predict({'input1': tokens_tensor_test_anc, 'input2': contexts_tensor_test_anc})
print(f1_score(np.argmax(preds_anc, axis=1), np.argmax(Y_test_anc, axis=1), average='micro'))
print(f1_score(np.argmax(preds_anc, axis=1), np.argmax(Y_test_anc, axis=1), average='macro'))
print(f1_score(np.argmax(preds_anc, axis=1), np.argmax(Y_test_anc, axis=1), average='weighted'))

0.7230971128608924
0.6771530270266309
0.7213731338246442


### 10%

In [ ]:
model_anc_10 = build_model()

In [ ]:
len(X_train_tokens_anc) * 0.1

177.8

In [ ]:
len(X_test_tokens_anc) * 0.1

76.2

In [ ]:
tokens_tensor_train_anc_10 = tf.convert_to_tensor(X_train_tokens_anc[:177])
tokens_tensor_test_anc_10 = tf.convert_to_tensor(X_test_tokens_anc[:76])
contexts_tensor_train_anc_10 = tf.convert_to_tensor(X_train_contexts_anc[:177])
contexts_tensor_test_anc_10 = tf.convert_to_tensor(X_test_contexts_anc[:76])

In [ ]:
model_anc_10.fit({'input1': tokens_tensor_train_anc_10, 'input2': contexts_tensor_train_anc_10}, 
               {'output': Y_train_anc[:177]}, batch_size=8, epochs=10)

Epoch 1/10
23/23 [==============================] - 9s 107ms/step - loss: 0.6242 - accuracy: 0.4633
Epoch 2/10
23/23 [==============================] - 2s 106ms/step - loss: 0.5663 - accuracy: 0.5198
Epoch 3/10
23/23 [==============================] - 2s 105ms/step - loss: 0.5386 - accuracy: 0.5932
Epoch 4/10
23/23 [==============================] - 2s 105ms/step - loss: 0.5076 - accuracy: 0.6271
Epoch 5/10
23/23 [==============================] - 2s 108ms/step - loss: 0.4690 - accuracy: 0.6836
Epoch 6/10
23/23 [==============================] - 2s 104ms/step - loss: 0.4172 - accuracy: 0.7684
Epoch 7/10
23/23 [==============================] - 3s 111ms/step - loss: 0.3797 - accuracy: 0.7797
Epoch 8/10
23/23 [==============================] - 2s 106ms/step - loss: 0.3457 - accuracy: 0.8079
Epoch 9/10
23/23 [==============================] - 2s 107ms/step - loss: 0.3078 - accuracy: 0.8757
Epoch 10/10
23/23 [==============================] - 3s 111ms/step - loss: 0.2683 - accuracy: 0.8983

In [ ]:
model_anc_10.evaluate({'input1': tokens_tensor_test_anc_10, 'input2': contexts_tensor_test_anc_10}, 
               {'output': Y_test_anc[:76]})

3/3 [==============================] - 1s 46ms/step - loss: 0.3939 - accuracy: 0.7237


[0.3938930332660675, 0.7236841917037964]

In [ ]:
model_anc_10.evaluate({'input1': tokens_tensor_test_anc, 'input2': contexts_tensor_test_anc}, 
               {'output': Y_test_anc})

24/24 [==============================] - 1s 49ms/step - loss: 0.4407 - accuracy: 0.6811


[0.44073590636253357, 0.6811023354530334]

In [ ]:
preds_anc = model_anc_10.predict({'input1': tokens_tensor_test_anc_10, 'input2': contexts_tensor_test_anc_10})
print(f1_score(np.argmax(preds_anc, axis=1), np.argmax(Y_test_anc[:76], axis=1), average='micro'))
print(f1_score(np.argmax(preds_anc, axis=1), np.argmax(Y_test_anc[:76], axis=1), average='macro'))
print(f1_score(np.argmax(preds_anc, axis=1), np.argmax(Y_test_anc[:76], axis=1), average='weighted'))

0.7236842105263158
0.6627346300548914
0.7175054311277221


In [ ]:
preds_anc = model_anc_10.predict({'input1': tokens_tensor_test_anc, 'input2': contexts_tensor_test_anc})
print(f1_score(np.argmax(preds_anc, axis=1), np.argmax(Y_test_anc, axis=1), average='micro'))
print(f1_score(np.argmax(preds_anc, axis=1), np.argmax(Y_test_anc, axis=1), average='macro'))
print(f1_score(np.argmax(preds_anc, axis=1), np.argmax(Y_test_anc, axis=1), average='weighted'))

0.6811023622047244
0.6395481678106575
0.6805693329009728
